In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName("Collinear Points").setMaster("local[4]")
sc = SparkContext(conf=conf)

In [3]:
rdd = sc.textFile("data.txt")

In [4]:
def func(x):
    arr = x.split()
    return tuple([int(el) for el in arr])

def filt(x):
    print(x)
    return x[0] != x[1]

def findslope(x):
    if x[1][0] - x[0][0] == 0:
        slope = "inf"
    else:
        slope = 1.0 * (x[1][1] - x[0][1])/(x[1][0] - x[0][0])
    return ((x[0], slope), (x[1]))

def returnpoints(x):
#     print(x[0])
    x[1].append(x[0][0])
    arr = sorted(x[1])
    return tuple(arr)
#     return list(x[1]).append(x[0][0])

def removedups(x, y):
    return set((x,y))

def get_collinear():
    rdd1 = rdd.map(func)
    rdd1 = rdd1.cartesian(rdd1)
    rdd1 = rdd1.filter(filt).map(findslope).groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)
    rdd1 = rdd1.map(returnpoints)
    res = set(rdd1.collect())
    #res = rdd1.collect()
    return res

In [5]:
res = get_collinear()